# Logistic Regression model

In [ ]:
%load_ext autotime

In [ ]:
import torch
from transformers import BertTokenizer
import json
from collections import Counter
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
import copy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

In [ ]:
import pandas as pd
df_annotations=  pd.read_csv('full_cleaned_dataset.csv')
df_annotations.head() # to display the first 5 lines of loaded data

In [ ]:
def TFIDF(data, max_feat, n_gram):
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(data)
    X= vect.fit_transform(data)
    X=pd.DataFrame(X.toarray(), columns= vect.get_feature_names())
    return X

In [ ]:
def BOW(data, ngram, max_features=100):
    vect=CountVectorizer(max_features, ngram_range=ngram)
    vect.fit(data)
    X_review=vect.transform(data)

    X_df=pd.DataFrame(X_review.toarray(), columns=vect.get_feature_names())
    return X_df

In [ ]:
def y_set(dataset_column):
    y= dataset_column
    y = y.replace('NEUTRAL','1')
    y = y.replace('POSITIVE','2')
    y = y.replace('NEGATIVE','0')
    for i in y:
        i=int(i)
    return y

In [ ]:
def cf_mat(y_test, y_pred):
    mat=confusion_matrix(y_test, y_pred)/len(y_test)
    return sns.heatmap(mat, annot=True)

In [ ]:
def logreg(X, y_inputkolom, reg_term=1):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Build a logistic regression
    log_reg = LogisticRegression(max_iter=500, C= reg_term).fit(X_train, y_train)

    # Predict the labels 
    y_predict = log_reg.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', log_reg.score(X_test, y_test))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    
    
    #return y_predict, y_test, log_reg

In [ ]:
def logreg_val(X,y_inputkolom):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    log_reg = LogisticRegression(max_iter=500).fit(X_train, y_train)

    # Predict the labels 
    y_predict = log_reg.predict(X_val)
    acc_score= log_reg.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    return acc_score

In [ ]:
def logreg_val_reg(X,y_inputkolom, reg_term, max_it=500):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    log_reg = LogisticRegression(max_iter=max_it, C=reg_term).fit(X_train, y_train)

    # Predict the labels 
    y_predict = log_reg.predict(X_val)
    acc_score= log_reg.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

# TFIDF

## Plain text

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.text.values.astype('U'),i,(1,1))
    acc_score=logreg_val(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    X=TFIDF(df_annotations.text.values.astype('U'),6000,i)
    logreg_val(X,df_annotations['label'])

optimizing the regularization term

In [ ]:
plotje=[]
max_feat_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
X= TFIDF(df_annotations.text.values.astype('U'),6000,(1,1))
for i in max_feat_list:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

TFIDF and then logreg with optimal parameters

In [ ]:
%%time

X=TFIDF(df_annotations.text.values.astype('U'),6000,(1,1))
logreg(X,df_annotations['label'], 0.9)

## Stemming

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),i,(1,1))
    acc_score=logreg_val(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    X=TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),6000,i)
    logreg_val(X,df_annotations['label'])

optimizing the regularization term

In [ ]:
plotje=[]
max_feat_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
X= TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),6000,(1,1))
for i in max_feat_list:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

TFIDF with optimal parameters with extra preprocessing

In [ ]:
%%time
X= TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),6000, (1,1))
logreg(X, df_annotations['label'], 0.7)

## Lemmatization

In [ ]:
plotje=[]
max_feat_list=[100,300,500,750 ,1000, 1100, 1500, 2000, 2500, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),i,(1,1))
    acc_score=logreg_val(X, df_annotations['label'])
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    X=TFIDF(df_annotations.processed_annotations_joined.values.astype('U'),10000,i)
    logreg_val(X,df_annotations['label'])

In [ ]:
plotje=[]
max_feat_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in max_feat_list:
    print('the regularisation term is', i)
    X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.astype('U'),10000,(1,1))
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
X= TFIDF(df_annotations.processed_annotations_lemmatized_joined.values.astype('U'),10000, (1,1))
logreg(X, df_annotations['label'],0.5)

## Word2Vec selftrained

In [ ]:
def sent(data_column):
    sentences=[]
    for i in range(len(df_annotations['text'])):
        words_in_sentence=df_annotations['text'][i].split()
        sentences.append(words_in_sentence)
    return sentences

In [ ]:
def W2V(sent, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025):
    model= Word2Vec(sentences=sent, vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in range(len(df_annotations['text'])):
        l=[]
        for word in df_annotations['text'][i].split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

## Textified

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    X=W2V(sent(df_annotations['text']), i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

so we choose vector_size= 200

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    X=W2V(sent(df_annotations['text']), 200, i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

we kiezen window 15

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,50]
for i in min_c:
    X=W2V(sent(df_annotations['text']), 200, 15,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

dus we kiezen min_count 2

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V(sent(df_annotations['text']), 200, 15,2,16,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

dus sgd=1

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    X=W2V(sent(df_annotations['text']), 200, 15,2,16,1,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

Dus we kiezen 0.15

In [ ]:
plotje=[]
reg_term=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in reg_term:
    print('the regularisation term is', i)
    X=W2V(sent(df_annotations['text']), 200, 15,2,16,1,0.15)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=W2V(sent(df_annotations['text']), 200, 15,2,16,1,0.15)
acc_score=logreg(X, df_annotations['label'], 0.2)

## Lemmatized

In [ ]:
def W2V_lem(sent, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025):
    model= Word2Vec(sentences=sent, vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in range(len(df_annotations['processed_annotations_lemmatized_joined'])):
        l=[]
        if type(df_annotations['processed_annotations_lemmatized_joined'][i])== float:
            l.append([0]*vector_s)
        else:
            for word in df_annotations['processed_annotations_lemmatized_joined'][i].split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20,30,50]
for i in min_c:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, 12,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, 12,5,16,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1, 1.2]
for i in alphas:
    X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, 12,5,16,1,i)
    acc_score=logreg_val_reg(X, df_annotations['label'], 1, 1000)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
plotje=[]
reg_term=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, 12,5,16,1,0.1)
for i in reg_term:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=W2V_lem(sent(df_annotations['processed_annotations_lemmatized_joined']), 300, 12,5,16,1,0.1)
acc_score=logreg(X, df_annotations['label'], 0.2)

# Word2Vec using pretrained embeddings

In [ ]:
import gensim.downloader as api
corpus = api.load('text8')
model = Word2Vec(corpus)

In [ ]:
import gensim.downloader as api
corpus = api.load('text8')
model = Word2Vec(corpus)

In [ ]:
import gensim

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [ ]:
def W2V_pretrained_emb(data):
    Word2Vec_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                if word in model.key_to_index:
                    l.append(list(model.get_vector(word)))
                else:
                    l.append([0]*300)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

## Textified

In [ ]:
plotje=[]
reg_term=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in reg_term:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['text'])
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=W2V_pretrained_emb(df_annotations['text'])
acc_score=logreg(X, df_annotations['label'], 0.1)

## Lemmatized

In [ ]:
plotje=[]
reg_term=[0.05, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in reg_term:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=logreg(X, df_annotations['label'], 0.05)

# Fasttext using pretrained embeddings

In [ ]:
#https://fasttext.cc/docs/en/support.html#building-fasttext-python-module
#https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
import fasttext
import fasttext.util

In [ ]:
fasttext.util.download_model('nl', if_exists='ignore')  # English

In [ ]:
model = fasttext.load_model('cc.nl.300.bin')

In [ ]:
def fasttext_pretrained_emb(data):
    fasttext_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                l.append(list(model.get_word_vector(word)))
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        fasttext_text_av.append(avg)
    X=pd.DataFrame(fasttext_text_av)
    return X

## Textified

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
acc_score=logreg_val_reg(X, df_annotations['label'], 0.05)

In [ ]:
plotje=[]
reg_term=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
X=fasttext_pretrained_emb(df_annotations['text'])
for i in reg_term:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
reg_term=[1,1.2,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3]
X=fasttext_pretrained_emb(df_annotations['text'])
for i in reg_term:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
acc_score=logreg(X, df_annotations['label'], 2.2)

## Lemmatized

In [ ]:
plotje=[]
reg_term=[0.05, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1, 1.5, 2, 2.5, 3]
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in reg_term:
    print('the regularisation term is', i)
    acc_score=logreg_val_reg(X, df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(reg_term, plotje)
plt.show()

In [ ]:
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=logreg(X, df_annotations['label'], 1)